In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from root_numpy import root2array, rec2array

branch_names = '''momentumx, momentumy,momentumz,momentum,energy,position,MvdDEDX,MvdHits,SttMeanDEDX,SttHits,GemHits,TofStopTime,
TofM2,TofTrackLength,TofQuality,TofBeta,DrcThetaC,DrcQuality,
DiscThetaC,DiscQuality,
EmcRawEnergy,EmcCalEnergy,EmcQuality,EmcNumberOfCrystals,EmcNumberOfBumps,EmcModule,
EmcZ20,EmcZ53,EmcLat,EmcE1,EmcE9,EmcE25,MuoQuality,MuoIron,MuoMomentumIn,MuoNumberOfLayers,MuoModule,MuoHits,
DegreesOfFreedom,ChiSquared'''.split(",")

branch_names = [c.strip() for c in branch_names]
branch_names = list(branch_names)
###################################################################################

electrons = root2array('/home/wesmail/Downloads/150k/electron_150k_tree.root', 't1', branch_names)
electrons = rec2array(electrons)

pions = root2array('/home/wesmail/Downloads/150k/pion_150k_tree.root', 't1', branch_names)
pions = rec2array(pions)

muons = root2array('/home/wesmail/Downloads/150k/muon_150k_tree.root', 't1', branch_names)
muons = rec2array(muons)

kaons = root2array('/home/wesmail/Downloads/150k/kaon_150k_tree.root', 't1', branch_names)
kaons = rec2array(kaons)

anti_p = root2array('/home/wesmail/Downloads/150k/proton_150k_tree.root', 't1', branch_names)
anti_p = rec2array(anti_p)

###################################################################################
X = np.concatenate((electrons, pions, muons, kaons, anti_p))
y = np.concatenate(( np.zeros(electrons.shape[0]),np.ones(pions.shape[0]), (2*np.ones(muons.shape[0])), (3*np.ones(kaons.shape[0])), (4*np.ones(anti_p.shape[0])) ))
df = pd.DataFrame(np.hstack((X, y.reshape(y.shape[0], -1))),columns=branch_names+['temp'])

# added features
#df['E1/E9'] = df.loc[:,'EmcE1']/df.loc[:,'EmcE9']
#df['E9/E25'] = df.loc[:,'EmcE9']/df.loc[:,'EmcE25']
#df['E1/E25'] = df.loc[:,'EmcE1']/df.loc[:,'EmcE25']
df['E/p'] = df.loc[:,'EmcCalEnergy']/df.loc[:,'momentum']
df['labels'] = df.loc[:,'temp']
df = df.drop(['temp'], axis=1)

df = df.dropna()
print(len(df.columns))

Welcome to ROOTaaS 6.06/02
42


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict

X_train,X_test, y_train,y_test = train_test_split(df.iloc[:,0:-1], df.iloc[:,-1], stratify=df.iloc[:,-1], test_size=0.30, random_state=0)
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.fit_transform(X_test)

bdt = RandomForestClassifier(n_estimators=10, max_depth=15, max_features=35, random_state=0, n_jobs=4)
%time model = bdt.fit(X_train, y_train)
print('accuracy on training set: %.2f' % (model.score(X_train, y_train)*100) + '%')
print('accuracy on testing set : %.2f' % (model.score(X_test, y_test)*100) + '%')

import matplotlib.pyplot as plt
%matplotlib tk

CPU times: user 1min 55s, sys: 43.2 ms, total: 1min 55s
Wall time: 33.8 s
accuracy on training set: 89.25%
accuracy on testing set : 85.95%


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from root_numpy import root2array, rec2array

branch_names = '''momentumx, momentumy,momentumz,momentum,energy,position,MvdDEDX,MvdHits,SttMeanDEDX,SttHits,GemHits,TofStopTime,
TofM2,TofTrackLength,TofQuality,TofBeta,DrcThetaC,DrcQuality,
DiscThetaC,DiscQuality,
EmcRawEnergy,EmcCalEnergy,EmcQuality,EmcNumberOfCrystals,EmcNumberOfBumps,EmcModule,
EmcZ20,EmcZ53,EmcLat,EmcE1,EmcE9,EmcE25,MuoQuality,MuoIron,MuoMomentumIn,MuoNumberOfLayers,MuoModule,MuoHits,
DegreesOfFreedom,ChiSquared'''.split(",")

branch_names = [c.strip() for c in branch_names]
branch_names = list(branch_names)
###################################################################################

electronsp = root2array('/home/wesmail/Downloads/150k/plus/Pelectron_theta_150k_tree.root', 't1', branch_names)
electronsp = rec2array(electronsp)

pionsp = root2array('/home/wesmail/Downloads/150k/plus/Ppion_theta_150k_tree.root', 't1', branch_names)
pionsp = rec2array(pionsp)

muonsp = root2array('/home/wesmail/Downloads/150k/plus/Pmuon_theta_150k_tree.root', 't1', branch_names)
muonsp = rec2array(muonsp)

kaonsp = root2array('/home/wesmail/Downloads/150k/plus/Pkaon_theta_150k_tree.root', 't1', branch_names)
kaonsp = rec2array(kaonsp)

anti_pp = root2array('/home/wesmail/Downloads/150k/plus/Pproton_theta_150k_tree.root', 't1', branch_names)
anti_pp = rec2array(anti_pp)

###################################################################################
Xp = np.concatenate((electronsp, pionsp, muonsp, kaonsp, anti_pp))
yp = np.concatenate(( np.zeros(electronsp.shape[0]),np.ones(pionsp.shape[0]), (2*np.ones(muonsp.shape[0])), (3*np.ones(kaonsp.shape[0])), (4*np.ones(anti_pp.shape[0])) ))
dfp = pd.DataFrame(np.hstack((Xp, yp.reshape(yp.shape[0], -1))),columns=branch_names+['temp'])

# added features
dfp['E1/E9'] = dfp.loc[:,'EmcE1']/dfp.loc[:,'EmcE9']
dfp['E9/E25'] = dfp.loc[:,'EmcE9']/dfp.loc[:,'EmcE25']
dfp['E1/E25'] = dfp.loc[:,'EmcE1']/dfp.loc[:,'EmcE25']
dfp['E/p'] = dfp.loc[:,'EmcCalEnergy']/dfp.loc[:,'momentum']
dfp['labels'] = dfp.loc[:,'temp']
dfp = dfp.drop(['temp'], axis=1)

dfp = dfp.dropna()

In [7]:
model.classes_
from sklearn.metrics import confusion_matrix
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns

figcm, ax = plt.subplots()
y_predicted = model.predict(X_test)
cm = confusion_matrix(y_test, y_predicted)
cm = np.round((cm.astype('float') / cm.sum(axis=1)), decimals=2)
sns.set(font_scale=2.0)
sns.heatmap(cm, square=True, annot=True, annot_kws={"size": 22}, cmap='Blues')
#sns.heatmap(cm, square=True, annot=True, cbar=False, annot_kws={"size": 20})
classes=['$e^{-}$', '$\pi^{-}$', '$\mu^{-}$', '$k^{-}$', '$\overline{p}$'] # 0 1 2 3 4 output of 'model.classes_'
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=90)
ax.set_xticks(np.arange(len(classes))+0.5, minor=False)
plt.yticks(tick_marks, classes)
ax.set_yticks(np.arange(len(classes))+0.5, minor=False)
plt.xlabel('Predication', horizontalalignment = 'center',  fontsize=22)
plt.ylabel('True Species',  fontsize=22)
plt.title('Particles',  fontsize=22)

In [6]:
# Compute ROC curve and ROC area for each class
from sklearn.metrics import roc_curve, auc
y_score = model.predict_proba(X_test)
from sklearn.preprocessing import label_binarize
y_test = label_binarize(y_test, classes=[0, 1, 2, 3, 4])
n_classes = y_test.shape[1]
import matplotlib.pyplot as plt
%matplotlib tk

lw=2
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

from itertools import cycle

plt.plot(fpr[0], tpr[0], color='aqua', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$e^{-}$',roc_auc[0]))
plt.plot(fpr[1], tpr[1], color='darkorange', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$\pi^{-}$',roc_auc[1]))
plt.plot(fpr[2], tpr[2], color='cornflowerblue', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$\mu^{-}$',roc_auc[2]))
plt.plot(fpr[3], tpr[3], color='black', lw=lw, label='ROC for {0} (area = {1:0.2f})'.format('$k^{-}$',roc_auc[3]))
plt.plot(fpr[4], tpr[4], color='green', lw=lw, label='ROC for {0}   (area = {1:0.2f})'.format('$\overline{p}$',roc_auc[4]))

plt.xlim([0.0, 0.5])
plt.ylim([0.5, 1.0])
plt.xlabel('False Positive Rate', fontsize=22)
plt.ylabel('True Positive Rate', fontsize=22)
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right', fontsize=22)    